<a href="https://colab.research.google.com/github/zhenya-mamenko/mini-ML-piscine/blob/master/first_steps_with_tensor_flow_with_tf2_and_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2017 Google LLC., 2019 Zhenya Mamenko
This notebook based on [First Steps with TensorFlow](https://colab.research.google.com/notebooks/mlcc/first_steps_with_tensor_flow.ipynb?utm_source=zhenya_mamenko&utm_campaign=colab-external&utm_medium=referral&utm_content=firststeps-colab&hl=en) exercise from [Google Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/).

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# First Steps with TensorFlow

**Learning Objectives:**
  * Learn fundamental TensorFlow 2.0 and Keras concepts
  * Use the `Sequential` model with `Dense` layer and `linear` activation function to predict median housing price, at the granularity of city blocks, based on one input feature
  * Evaluate the accuracy of a model's predictions using Root Mean Squared Error (RMSE)
  * Improve the accuracy of a model by tuning its hyperparameters

The [data](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) is based on 1990 census data from California.

## Setup
In this first cell, we'll load the `tensorflow 2.0.0-beta1` and other necessary libraries.

In [0]:
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import logging
from IPython.display import display
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
logging.getLogger('tensorflow').disabled = True

!pip install -q tensorflow==2.0.0-beta1

import tensorflow as tf
from tensorflow.keras import layers

Make sure that our kernel use version 2.0 of TensorFlow. If not, reload kernel with `Ctrl+M .`

In [0]:
print(tf.__version__)

Next, we'll load our data set.

In [0]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

We'll randomize the data, just to be sure not to get any pathological ordering effects that might harm the performance of Stochastic Gradient Descent. Additionally, we'll scale `median_house_value` to be in units of thousands, so it can be learned a little more easily with learning rates in a range that we usually use.

In [4]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
16128,-122.5,37.7,49.0,1207.0,255.0,658.0,220.0,4.1,228.6
4151,-118.0,34.1,37.0,1568.0,371.0,1618.0,350.0,3.0,129.4
634,-117.0,32.6,14.0,2796.0,476.0,1466.0,464.0,5.2,213.7
16317,-122.5,37.8,45.0,1673.0,377.0,1078.0,393.0,3.3,272.3
5895,-118.2,33.8,47.0,2347.0,437.0,1219.0,420.0,5.3,209.9
...,...,...,...,...,...,...,...,...,...
5125,-118.1,34.6,29.0,2370.0,475.0,1746.0,483.0,3.7,113.5
9124,-119.0,35.3,23.0,4870.0,965.0,2717.0,928.0,2.6,70.0
8654,-118.5,34.4,14.0,15348.0,2366.0,7087.0,2169.0,6.3,237.7
6094,-118.2,34.2,48.0,1473.0,294.0,807.0,296.0,3.4,306.3


## Examine the Data

It's a good idea to get to know your data a little bit before you work with it.

We'll print out a quick summary of a few useful statistics on each column: count of examples, mean, standard deviation, max, min, and various quantiles.

In [0]:
california_housing_dataframe.describe()

## Build the First Model

In this exercise, we'll try to predict `median_house_value`, which will be our label (sometimes also called a target). We'll use `total_rooms` as our input feature.

**NOTE:** Our data is at the city block level, so this feature represents the total number of rooms in that block.

To train our model, we'll use the [Sequential](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Sequential) interface provided by the TensorFlow 2.0 [Keras](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras) API. This is high-level API for TensorFlow.

### Step 1: Define Features

There are two main types of data we'll use in this and future exercises:

* **Categorical Data**: Data that is textual. In this exercise, our housing data set does not contain any categorical features, but examples you might see would be the home style, the words in a real-estate ad.

* **Numerical Data**: Data that is a number (integer or float) and that you want to treat as a number. As we will discuss more later sometimes you might want to treat numerical data (e.g., a postal code) as if it were categorical.

In Keras, we don't need any special preparations of our data, just get NumPy arrays from dataset.

To start, we're going to use just one numeric input feature, `total_rooms`. The following code pulls the `total_rooms` data from our `california_housing_dataframe`:

In [0]:
# Define the input feature: total_rooms.
features = california_housing_dataframe[["total_rooms"]].values

**NOTE:** The shape of our `total_rooms` data is a one-dimensional array (a list of the total number of rooms for each block).

### Step 2: Define the Label

Next, we'll define our label, which is `median_house_value`. Again, we can pull it from our `california_housing_dataframe`:

In [0]:
# Define the label.
labels = california_housing_dataframe["median_house_value"].values

### Step 3: Configure the Model

 Root Mean Squared Error (RMSE) is not part of [built-in loss functions](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses) and we have to implement it.

In [0]:
def rmse(y_true, y_pred):
  mse = tf.keras.losses.MeanSquaredError()
  return tf.math.sqrt(mse(y_true, y_pred))

Next, we'll configure a simple Keras model using `Sequential` class. We'll use only one [Dense](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense) layer and than we'll train this model using the [Stohastic Gradient Descent](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/optimizers/SGD) optimizer `SGD`. The `learning_rate` argument controls the size of the gradient step.

**NOTE:** To be safe, we also apply [gradient clipping](https://developers.google.com/machine-learning/glossary/#gradient_clipping) to our optimizer via `clipnorm` parameter. Gradient clipping ensures the magnitude of the gradients do not become too large during training, which can cause gradient descent to fail. 

In [0]:
# Use linear activation function to emulate LinearRegressor estimator from TF1
# You can provide `input_dim` argument, which convert to `input_shape` as `(input_dim, )`
# Note that you can also omit the `input_shape` argument:
# In that case the model gets built the first time you call `fit` (or other
# training and evaluation methods).

model = tf.keras.models.Sequential([
    layers.Dense(1, activation='linear', bias_initializer='he_uniform')
])

# Use gradient descent as the optimizer for compiile the model and MSE metric.
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=5.0),
              loss=rmse,
              metrics=['mse'])


### Step 4: Train the Model

We can now call `fit()` on our `model` to train it, and to start, we'll train for 100 steps.

In [9]:
history = model.fit(features, labels, epochs=1, steps_per_epoch=100, batch_size=1, verbose=0).history
root_mean_squared_error = history["loss"][0]
mean_squared_error = history["mse"][0]
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 967686.625
Root Mean Squared Error (on training data): 538.384


**NOTE:** Training error measures how well your model fits the training data, but it **_does not_** measure how well your model **_generalizes to new data_**. In later exercises, you'll explore how to split your data to evaluate your model's ability to generalize.


Is this a good model? How would you judge how large this error is?

Mean Squared Error (MSE) can be hard to interpret, so we often look at Root Mean Squared Error (RMSE)
instead.  A nice property of RMSE is that it can be interpreted on the same scale as the original targets.

Let's compare the RMSE to the difference of the min and max of our targets:

In [0]:
min_house_value = california_housing_dataframe["median_house_value"].min()
max_house_value = california_housing_dataframe["median_house_value"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

Our error is too big concerning to the target values. Can we do better?

This is the question that nags at every model developer. Let's develop some basic strategies to reduce model error.

The first thing we can do is take a look at how well our predictions match our targets, in terms of overall summary statistics.

In [0]:
calibration_data = pd.DataFrame()
calibration_data["predictions"] = model.predict_on_batch(california_housing_dataframe["total_rooms"].values).flatten()
calibration_data["targets"] = pd.Series(labels)
calibration_data.describe()

Okay, maybe this information is helpful. How does the mean value compare to the model's RMSE? How about the various quantiles?

We can also visualize the data and the line we've learned.  Recall that linear regression on a single feature can be drawn as a line mapping input *x* to output *y*.

First, we'll get a uniform random sample of the data so we can make a readable scatter plot.

In [0]:
sample = california_housing_dataframe.sample(n=300)


Next, we'll plot the line we've learned, drawing from the model's bias term and feature weight, together with the scatter plot. The line will show up red.

In [0]:
# Get the min and max total_rooms values.
x_0 = sample["total_rooms"].min()
x_1 = sample["total_rooms"].max()

# Retrieve the final weight and bias generated during training.
weight, bias = [x.flatten()[0] for x in model.layers[0].get_weights()]

# Get the predicted median_house_values for the min and max total_rooms values.
y_0 = weight * x_0 + bias 
y_1 = weight * x_1 + bias

# Plot our regression line from (x_0, y_0) to (x_1, y_1).
plt.plot([x_0, x_1], [y_0, y_1], c='r')

# Label the graph axes.
plt.ylabel("median_house_value")
plt.xlabel("total_rooms")

# Plot a scatter plot from our data sample.
plt.scatter(sample["total_rooms"], sample["median_house_value"])

# Display graph.
plt.show()

This initial line looks way off.  See if you can look back at the summary stats and see the same information encoded there.

Together, these initial sanity checks suggest we may be able to find a much better line.

## Tweak the Model Hyperparameters
For this exercise, we've write function for training model. You can call the function with different parameters to see the effect.

In this function, we'll proceed in 10 epochs so that we can observe the model improvement at each epoch.

For each epoch, we'll graph training loss.  This may help you judge when a model is converged, or if it needs more iterations.
We'll also plot the feature weight and bias term values learned by the model over time.  This is another way to see how things converge.
To do this, we'll implement simple [Lambda](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/LambdaCallback) [callback](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks).

In [0]:
def fit_model(learning_rate=0.001, steps_per_epoch=100, batch_size=1, input_feature="total_rooms"):
  """Trains a linear regression model of one feature.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps_per_epoch: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `california_housing_dataframe`
      to use as input feature.
  """
  epochs = 10
  features = california_housing_dataframe[[input_feature]].values
  label = "median_house_value"
  labels = california_housing_dataframe[label].values
  
  model = tf.keras.models.Sequential([
    layers.Dense(1, activation='linear', bias_initializer='he_uniform')
  ])
  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=5.0),
              loss=rmse,
              metrics=['mse'])
  
  # Set up to plot the state of our model's line each epoch.
  def create_plt(feature, label, epochs=10):
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.title("Learned Line by Epoch")
    plt.ylabel(label)
    plt.xlabel(feature)
    sample = california_housing_dataframe.sample(n=500)
    plt.scatter(sample[feature], sample[label])
    colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, epochs)]
    return (colors,
            (sample[feature].min(), sample[feature].max()),
            (0, sample[label].max()))

  colors, x_min_max, y_min_max = create_plt(input_feature, label, epochs)

  def log(epoch, logs):
    root_mean_squared_error = logs["loss"]
    print("  epoch %02d : %0.2f" % (epoch, root_mean_squared_error))

    weight, bias = [x.flatten()[0] for x in model.layers[0].get_weights()]
    # Apply some math to ensure that the data and line are plotted neatly.
    y_extents = np.array(y_min_max)
    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      x_min_max[1]),
                           x_min_max[0])
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[epoch]) 

  model_callback = tf.keras.callbacks.LambdaCallback(
      on_epoch_end=lambda epoch, logs: log(epoch, logs))
  
  print("Train model...")
  print("RMSE (on training data):")
  history = model.fit(features,
            labels,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            batch_size=batch_size,
            callbacks=[model_callback],
            verbose=0).history
  print("Model training finished.")

  # Output a graph of loss metrics over epochs.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Epochs')
  plt.title("Root Mean Squared Error vs. Epochs")
  plt.tight_layout()
  plt.plot(history["loss"])
  
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = model.predict_on_batch(features).flatten()
  calibration_data["targets"] = pd.Series(labels)
  display(calibration_data.describe())

## Task 1:  Achieve an RMSE of 180 or Below

Tweak the model hyperparameters to improve loss and better match the target distribution.
If, after 5 minutes or so, you're having trouble beating a RMSE of 180, check the solution for a possible combination.

In [0]:
fit_model(learning_rate=0.001,
          steps_per_epoch=10,
          batch_size=1,
          input_feature="total_rooms")

### Solution

Click below for one possible solution.

In [0]:
fit_model(learning_rate=0.001,
          steps_per_epoch=50,
          batch_size=10,
          input_feature="total_rooms",
          clipnorm=5.0)

This is just one possible configuration; there may be other combinations of settings that also give good results. Note that in general, this exercise isn't about finding the *one best* setting, but to help build your intutions about how tweaking the model configuration affects prediction quality.

### Is There a Standard Heuristic for Model Tuning?

This is a commonly asked question. The short answer is that the effects of different hyperparameters are data dependent. So there are no hard-and-fast rules; you'll need to test on your data.

That said, here are a few rules of thumb that may help guide you:

 * Training error should steadily decrease, steeply at first, and should eventually plateau as training converges.
 * If the training has not converged, try running it for longer.
 * If the training error decreases too slowly, increasing the learning rate may help it decrease faster.
   * But sometimes the exact opposite may happen if the learning rate is too high.
 * If the training error varies wildly, try decreasing the learning rate.
   * Lower learning rate plus larger number of steps or larger batch size is often a good combination.
 * Very small batch sizes can also cause instability.  First try larger values like 100 or 1000, and decrease until you see degradation.

Again, never go strictly by these rules of thumb, because the effects are data dependent.  Always experiment and verify.

## Task 2: Try a Different Feature

See if you can do any better by replacing the `total_rooms` feature with the `population` feature.

Don't take more than 5 minutes on this portion.

In [0]:
fit_model(learning_rate=0.05,
          steps_per_epoch=10,
          batch_size=1,
          input_feature="population")

### Solution

Click below for one possible solution.

In [0]:
fit_model(learning_rate=0.001,
          steps_per_epoch=100,
          batch_size=10,
          input_feature="population",
          clipnorm=5.0)